#First test

In [ ]:
# Import basic libraries
import pandas as pd
import numpy as np

In [ ]:
!pip install datasets transformers huggingface_hub


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Getting the data

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from transformers import AutoTokenizer, DistilBertModel
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import load_dataset

train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataScience/train.csv")
to_predict = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataScience/test.csv")

In [ ]:
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.9/830.9 kB 30.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.7 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16321 sha256=62f48bedc689b6b87669d82a2bcf57aa0bdbad593108c861c6c88c3e64f28607
  Stored in directory: /root/.cache/pip/wheels/2b/5e/f2/3931524f702ffd03309e96d35ee2fbf9c61c27377511ee8d4c
Successfully built swifter


In [ ]:
from tqdm.auto import tqdm 
tqdm.pandas()

from bs4 import BeautifulSoup
import re
import pandas as pd
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Using swifter for parallelization
import swifter

# Using regex pattern for non-alphanumeric removal
pattern = re.compile('[^\w\s]')

# Optimized preprocessing function
def preprocess_text(text):
    # remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # remove non-alphanumeric characters and convert to lower case
    text = pattern.sub('', text).lower()
    
    # remove stop words using Spacy
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc if not token.is_stop])
    
    return text

# Applying optimized function with swifter
train['review'] = train['review'].swifter.apply(preprocess_text)
to_predict['review'] = to_predict['review'].swifter.apply(preprocess_text)

Pandas Apply:   0%|          | 0/14911 [00:00<?, ?it/s]

<ipython-input-8-dee030084a7b>:22: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


Pandas Apply:   0%|          | 0/35089 [00:00<?, ?it/s]

<ipython-input-8-dee030084a7b>:22: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [ ]:
train['review'] = train['review'].apply(lambda x: " ".join([token.lemma_ for token in nlp(x)]))

KeyboardInterrupt: ignored

In [ ]:
train.to_csv('/content/clean_train.csv', sep='|')
to_predict.to_csv('/content/clean_test.csv', sep="|")

In [ ]:
train = train.replace({"response": {1:0, 2:1}})

In [ ]:
to_predict.head()

,review
0,petter mattei love time money visually stunnin...
1,phil alien one quirky film humour based around...
2,awful film must real stinker nominated golden ...
3,first let get thing straight anime fan always ...
4,taut organically gripping edward dmytryk cross...


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased").cuda()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenized = train['review'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True)))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(14911, 512)

In [ ]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

In [ ]:
with torch.no_grad():
    last_hidden_states = model(input_ids.cuda(), attention_mask=attention_mask.cuda())

OutOfMemoryError: ignored

In [ ]:
from transformers import pipeline
from tqdm import tqdm

# Define batch size
batch_size = 32

# Initialize sentiment analysis pipeline with batching option
sentiment_model = pipeline(model="leonardosaveri/dataScienceChallenge", 
                           tokenizer="leonardosaveri/dataScienceChallenge", 
                           device=0 if torch.cuda.is_available() else -1, 
                           max_length=512, 
                           truncation=True, 
                           batch_size=batch_size)

# Predict sentiment in batches and display progress bar
predictions = []
with tqdm(total=len(to_predict), desc="Predicting...") as pbar:
    for i in range(0, len(to_predict), batch_size):
        batch = to_predict.review.tolist()[i:i+batch_size]
        batch_pred = sentiment_model(batch)
        predictions.extend(batch_pred)
        pbar.update(len(batch))

Predicting...:   1%|          | 320/35089 [00:04<03:34, 161.83it/s]/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Predicting...: 100%|██████████| 35089/35089 [03:05<00:00, 189.47it/s]


In [ ]:
pred = []

for i in predictions:
    if i['label'] == 'LABEL_0':
        pred.append(1)
    else:
        pred.append(2)

In [ ]:
with open("/content/pred_test.txt", "w") as file:
    for element in pred:
        file.write(str(element) + "\n")

In [ ]:
import torch
torch.cuda.is_available()


train = train.rename_column("response", "labels")
train = train.rename_column('review', 'text')
train = train['train']

# Convert to pandas DataFrame
df = pd.DataFrame(train)

# Replace labels
df['labels'] = df['labels'].replace({1: 0, 2: 1})
df.to_csv('/content/train.csv')

train = load_dataset("csv", data_files='/content/train.csv')


from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


train = train['train']


train = train.train_test_split(test_size=0.2)


small_train_dataset = train["train"]
small_test_dataset = train["test"]


def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)
 
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)


from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}


from huggingface_hub import notebook_login
notebook_login()


from transformers import TrainingArguments, Trainer
 
repo_name = "dataScienceChallenge"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


trainer.train()

trainer.push_to_hub()


#2

In [ ]:
!pip install datasets transformers huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DataScience/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DataScience/test.csv')

In [ ]:
!pip install swifter

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

from bs4 import BeautifulSoup
import re
import pandas as pd
import spacy
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import nltk
nltk.download('stopwords')

import swifter

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

pattern = re.compile('[^\w\s]')

stop_words = stopwords.words('english')

def preprocess_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()
    text = pattern.sub('', text).lower() 
    text = " ".join([token.lemma_ for token in nlp(text) if not token.is_stop and token.lemma_ not in stop_words])
    return text 

train['review'] = train['review'].swifter.apply(preprocess_text)
test['review'] = test['review'].swifter.apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Pandas Apply:   0%|          | 0/14911 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/35089 [00:00<?, ?it/s]

In [ ]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
nltk.download('wordnet')

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

train['review'] = train['review'].swifter.apply(lemmatize_text)
test['review'] = test['review'].swifter.apply(lemmatize_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...


Pandas Apply:   0%|          | 0/14911 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/35089 [00:00<?, ?it/s]

In [ ]:
train = train.replace({"response": {1:0, 2:1}})

In [ ]:
train.to_csv('/content/drive/MyDrive/Colab Notebooks/DataScience/train_clean.csv')
test.to_csv('/content/drive/MyDrive/Colab Notebooks/DataScience/test_clean.csv')

In [ ]:
train['review'] = train['review'].apply(lambda x: " ".join(x))
test['review'] = test['review'].apply(lambda x: " ".join(x))

In [ ]:
from datasets import Dataset
train_ds = Dataset.from_pandas(train)
test_ds = Dataset.from_pandas(test)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True)


train_ds = train_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/14911 [00:00<?, ? examples/s]

Map:   0%|          | 0/35089 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="DSChallenge", push_to_hub=True,)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="DSChallenge", evaluation_strategy="epoch", push_to_hub=True)

In [ ]:
train_ds = train_ds.rename_column("review", "text")
train_ds = train_ds.rename_column("response", "label")
test_ds = test_ds.rename_column("review", "text")

### Trainer

Create a [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) object with your model, training arguments, training and test datasets, and evaluation function:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/leonardosaveri/DSChallenge into local empty directory.


Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
train_ds

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 14911
})

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.459100,No log


Epoch,Training Loss,Validation Loss
1,0.459100,No log
2,0.391100,No log
3,0.325200,No log


TrainOutput(global_step=5592, training_loss=0.4172348075670234, metrics={'train_runtime': 2967.1614, 'train_samples_per_second': 15.076, 'train_steps_per_second': 1.885, 'total_flos': 1.176974683941888e+16, 'train_loss': 0.4172348075670234, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/418M [00:00<?, ?B/s]

Upload file runs/Apr19_15-39-57_4e6d04944665/events.out.tfevents.1681918809.4e6d04944665.240.4:   0%|         …

To https://huggingface.co/leonardosaveri/DSChallenge
   8a1b3b2..a70acd9  main -> main

   8a1b3b2..a70acd9  main -> main

To https://huggingface.co/leonardosaveri/DSChallenge
   a70acd9..9224a42  main -> main

   a70acd9..9224a42  main -> main



'https://huggingface.co/leonardosaveri/DSChallenge/commit/a70acd9bb3f19712cc4ea26934a7a19454a2ab3c'

In [ ]:
import torch

In [ ]:
from transformers import pipeline
from tqdm import tqdm

# Define batch size
batch_size = 32

# Initialize sentiment analysis pipeline with batching option
sentiment_model = pipeline(model="huggingface.co/leonardosaveri/DSChallenge", 
                           tokenizer="leonardosaveri/DSChallenge", 
                           device=0 if torch.cuda.is_available() else -1, 
                           max_length=512, 
                           truncation=True, 
                           batch_size=batch_size)

# Predict sentiment in batches and display progress bar
predictions = []
with tqdm(total=len(test), desc="Predicting...") as pbar:
    for i in range(0, len(test), batch_size):
        batch = test['text'].tolist()[i:i+batch_size]
        batch_pred = sentiment_model(batch)
        predictions.extend(batch_pred)
        pbar.update(len(batch))

OSError: ignored

In [ ]:
pred = []

for i in predictions:
    if i['label'] == 'LABEL_0':
        pred.append(1)
    else:
        pred.append(2)

In [ ]:
with open("/content/pred_test.txt", "w") as file:
    for element in pred:
        file.write(str(element) + "\n")

# A MORE "NORMAL APPROACH

In [ ]:
import pandas as pd
import re
import nltk

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer

from tqdm.auto import tqdm 
tqdm.pandas()

nltk.download('all')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataScience/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataScience/test.csv")

In [ ]:
train = train.replace({"response": {1:0, 2:1}})

In [ ]:
# Removes the <br /> and other HTML tags
def remove_html(data):
    data = BeautifulSoup(data)
    return data.getText()

train.review = train.review.progress_map(remove_html)
test.review = test.review.progress_map(remove_html)

print(train.head())
test.head()

  0%|          | 0/14911 [00:00<?, ?it/s]

<ipython-input-23-33f6b0fccf39>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  data = BeautifulSoup(data)


  0%|          | 0/35089 [00:00<?, ?it/s]

   response                                             review
0         1  A wonderful little production. The filming tec...
1         0  Basically there's a family where a little boy ...
2         0  This show was an amazing, fresh & innovative i...
3         0  So im not a big fan of Boll's work but then ag...
4         0  Kind of drawn in by the erotic scenes, only to...


,review
0,"Petter Mattei's ""Love in the Time of Money"" is..."
1,Phil the Alien is one of those quirky films wh...
2,An awful film! It must have been up against so...
3,"First of all, let's get a few things straight ..."
4,"Taut and organically gripping, Edward Dmytryk'..."


In [ ]:
replacements = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "gonna": "going to",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'll": "he will",
    "he's": "he is",
    "how'd": "how did",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'll": "I will",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it would",
    "it'll": "it will",
    "it's": "it is",
    "Its" : "It is",
    "let's": "let us",
    "mightn't": "might not",
    "mustn't": "must not",
    "shan't": "shall not",
    "she'd": "she would",
    "she'll": "she will",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they'd": "they would",
    "wanna" : "want to",
    "We're" : "We are"
}

def remove_abb(review):
    for key, value in replacements.items():
        review = re.sub(r"{}".format(key), value, review)
    return review

train.review = train.review.progress_map(remove_abb)
test.review = test.review.progress_map(remove_abb)

print(train.head())
test.head()

  0%|          | 0/14911 [00:00<?, ?it/s]

  0%|          | 0/35089 [00:00<?, ?it/s]

   response                                             review
0         1  A wonderful little production. The filming tec...
1         0  Basically there is a family where a little boy...
2         0  This show was an amazing, fresh & innovative i...
3         0  So im not a big fan of Boll's work but then ag...
4         0  Kind of drawn in by the erotic scenes, only to...


,review
0,"Petter Mattei's ""Love in the Time of Money"" is..."
1,Phil the Alien is one of those quirky films wh...
2,An awful film! It must have been up against so...
3,"First of all, let us get a few things straight..."
4,"Taut and organically gripping, Edward Dmytryk'..."


In [ ]:
train.review = train.review.str.lower()
test.review = test.review.str.lower()

print(train.head())
test.head()

   response                                             review
0         1  a wonderful little production. the filming tec...
1         0  basically there is a family where a little boy...
2         0  this show was an amazing, fresh & innovative i...
3         0  so im not a big fan of boll's work but then ag...
4         0  kind of drawn in by the erotic scenes, only to...


,review
0,"petter mattei's ""love in the time of money"" is..."
1,phil the alien is one of those quirky films wh...
2,an awful film! it must have been up against so...
3,"first of all, let us get a few things straight..."
4,"taut and organically gripping, edward dmytryk'..."


In [ ]:
!pip install swifter

In [ ]:
stopwords_set = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
detokenizer = TreebankWordDetokenizer()


def clean_text(data):
    review_tokens = nltk.word_tokenize(data)
    review_tokens = set(review_tokens)
    review_tokens = [lemmatizer.lemmatize(words) for words in review_tokens if words.isalpha() and words not in stopwords_set]
    #review_tokens = [nltk.SnowballStemmer("english").stem(words) for words in review_tokens if words.isalpha() and words not in stopwords_set]
    return ' '.join(review_tokens)

train['review_clean'] = train.review.progress_map(clean_text)
test['review_clean'] = test.review.progress_map(clean_text)

print(train.head())
print(test.head())

In [ ]:
train.to_csv('/content/drive/MyDrive/Colab Notebooks/DataScience/clean_train.csv')
test.to_csv('/content/drive/MyDrive/Colab Notebooks/DataScience/clean_test.csv')

# FROM HERE AFTER CLEAN

In [ ]:
import pandas as pd
import re
import nltk
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer

from tqdm.auto import tqdm 
tqdm.pandas()

nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataScience/clean_train.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataScience/clean_test.csv")

In [ ]:
# Put everything into a single string
words  = ' '.join(train['review_clean'])
#words = words + ' '.join(test['review_clean'])
# Create a global tokenization
tokens = nltk.word_tokenize(words)

# Conversion to "text"
text = nltk.Text(tokens)
# Compute the most common words
fdist = nltk.FreqDist(text)

# Use pandas for organizing and displaying the results
df_words = pd.DataFrame(list(fdist.items()), columns = ["Word","Frequency"])
# Order words from the most frequent
df_words = df_words.sort_values(by = "Frequency", ascending = False)

# Dimension of the dataset
df_words.shape

(53001, 2)

In [ ]:
words_train = set(" ".join(train['review_clean']).split())
words_test = set(" ".join(test['review_clean']).split())

words = list(words_train.intersection(words_test))

In [ ]:
len(words)

40810

In [ ]:
max_tokens = df_words.query("Frequency > 1").shape[0]
max_tokens

30667

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Creation of a TDM TF-IDF with p = 500 words
vectorizer = TfidfVectorizer(max_features = max_tokens)#, vocabulary=words))
vec = vectorizer.fit(train.review_clean.append(test.review_clean, ignore_index=True))
X = vec.transform(train['review_clean'])
word_names = list(vectorizer.get_feature_names_out())

# Conversion to dataframe
X = pd.DataFrame(X.toarray())
# Renaming columns according to words
X.columns = word_names


X = X.values.astype(np.float32)
y = train['response'].values.astype(np.int32)

<ipython-input-60-3c6bf9e843d0>:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vec = vectorizer.fit(train.review_clean.append(test.review_clean, ignore_index=True))


In [ ]:
#@title What if we get only the intersection of words? PROBABLY NOT
train_words = set(nltk.word_tokenize(' '.join(train['review_clean'])))
test_words = set(nltk.word_tokenize(' '.join(test['review_clean'])))
common_words = list(train_words.intersection(test_words))

from sklearn.feature_extraction.text import TfidfVectorizer

# Creation of a TDM TF-IDF with p = number of common words
vectorizer = TfidfVectorizer(vocabulary=common_words)
vec = vectorizer.fit(train.review_clean.append(test.review_clean, ignore_index=True))
X = vec.transform(train['review_clean'])
word_names = list(vectorizer.get_feature_names_out())

# Conversion to dataframe
X = pd.DataFrame(X.toarray())
# Renaming columns according to words
X.columns = word_names

X = X.values.astype(np.float32)
y = train['response'].values.astype(np.int32)

In [ ]:
X.shape

##RandomForest

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

my_class = RandomForestClassifier(criterion='entropy', random_state= 42, verbose=True, n_jobs=-1)
#scoresD = cross_val_score(my_classM, X_train, y_train, cv=10, verbose=True, n_jobs=-1)
#print(scoresD.mean())

my_class.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   36.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.3min finished


RandomForestClassifier(criterion='entropy', n_jobs=-1, random_state=42,
                       verbose=True)

In [ ]:
accuracy_score(y_test, my_class.predict(X_test))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished


0.8431635388739946

##SVM

In [ ]:
from sklearn.svm import NuSVC

my_svm = NuSVC(nu = 0.2, verbose=True, kernel="linear", random_state=42)
#scores = cross_val_score(my_svmH, X_train, y_train, cv=10)
#print(scores.mean())

my_svm.fit(X_train,y_train)

[LibSVM]

NuSVC(kernel='linear', nu=0.2, random_state=42, verbose=True)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(my_svm.predict(X_test), y_test)

0.8666219839142091

##NN score of 583, reached around 89.ish accuracy


In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neural_network import MLPClassifier
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

clf = MLPClassifier(solver='adam', alpha=1e-6,
                    hidden_layer_sizes=(10, 2),
                    random_state=42, verbose=True, early_stopping=True)

# Fit the model to the training data
clf.fit(X_train, y_train)

Iteration 1, loss = 0.67629844
Validation score: 0.827124
Iteration 2, loss = 0.59856526
Validation score: 0.839046
Iteration 3, loss = 0.49217540
Validation score: 0.860656
Iteration 4, loss = 0.37728203
Validation score: 0.859911
Iteration 5, loss = 0.27890475
Validation score: 0.865872
Iteration 6, loss = 0.20643483
Validation score: 0.866617
Iteration 7, loss = 0.15520247
Validation score: 0.869598
Iteration 8, loss = 0.11883171
Validation score: 0.867362


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(alpha=1e-06, early_stopping=True, hidden_layer_sizes=(10, 2),
              random_state=42, verbose=True)

In [ ]:
clf.score(X_test, y_test)

0.8599195710455764

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = vec.transform(test['review_clean'])

# Conversion to dataframe
X = pd.DataFrame(X.toarray())
# Renaming columns according to words
X.columns = word_names


X = X.values.astype(np.float32)

In [ ]:
predictions = clf.predict(X)

In [ ]:
np.unique(predictions, return_counts=True)

(array([0, 1], dtype=int32), array([17749, 17340]))

In [ ]:
np.unique(y, return_counts=True)

(array([0, 1], dtype=int32), array([7523, 7388]))

In [ ]:
y_pred = [1 if i == 0 else 2 for i in predictions]

In [ ]:
with open("/content/pred_test_nn.txt", "w") as file:
    for element in y_pred:
        file.write(str(element) + "\n")

## NN , reached around 90ish accuraxy

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neural_network import MLPClassifier
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

clf = MLPClassifier(solver='adam', alpha=1e-6,
                    hidden_layer_sizes=(25, 10, 2),
                    random_state=42, verbose=True, early_stopping=True)

# Fit the model to the training data
clf.fit(X_train, y_train)

Iteration 1, loss = 0.78149405
Validation score: 0.502981
Iteration 2, loss = 0.65696010
Validation score: 0.703428
Iteration 3, loss = 0.48086492
Validation score: 0.885991
Iteration 4, loss = 0.27729084
Validation score: 0.888972
Iteration 5, loss = 0.15146718
Validation score: 0.885991
Iteration 6, loss = 0.08922663
Validation score: 0.878539
Iteration 7, loss = 0.05387552
Validation score: 0.871088
Iteration 8, loss = 0.03341359
Validation score: 0.865872
Iteration 9, loss = 0.02166759
Validation score: 0.861401
Iteration 10, loss = 0.01493028
Validation score: 0.861401
Iteration 11, loss = 0.01078481
Validation score: 0.857675
Iteration 12, loss = 0.00819986
Validation score: 0.855440
Iteration 13, loss = 0.00652505
Validation score: 0.857675
Iteration 14, loss = 0.00521942
Validation score: 0.854694
Iteration 15, loss = 0.00421858
Validation score: 0.853949
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(alpha=1e-06, early_stopping=True, hidden_layer_sizes=(25, 10, 2),
              random_state=42, verbose=True)

In [ ]:
clf.score(X_test, y_test)

0.9001340482573726

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

votingM = VotingClassifier(estimators=[('nn', clf), ('rf', my_class)],
                           voting = "soft", weights= None)

votingM.fit(X_train, y_train)
accuracy_score(votingM.predict(X_test), y_test )

Iteration 1, loss = 0.78149405
Validation score: 0.502981
Iteration 2, loss = 0.65696010
Validation score: 0.703428
Iteration 3, loss = 0.48086492
Validation score: 0.885991
Iteration 4, loss = 0.27729084
Validation score: 0.888972
Iteration 5, loss = 0.15146718
Validation score: 0.885991
Iteration 6, loss = 0.08922663
Validation score: 0.878539
Iteration 7, loss = 0.05387552
Validation score: 0.871088
Iteration 8, loss = 0.03341359
Validation score: 0.865872
Iteration 9, loss = 0.02166759
Validation score: 0.861401
Iteration 10, loss = 0.01493028
Validation score: 0.861401
Iteration 11, loss = 0.01078481
Validation score: 0.857675
Iteration 12, loss = 0.00819986
Validation score: 0.855440
Iteration 13, loss = 0.00652505
Validation score: 0.857675
Iteration 14, loss = 0.00521942
Validation score: 0.854694
Iteration 15, loss = 0.00421858
Validation score: 0.853949
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.3min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


0.8914209115281502

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = vec.transform(test['review_clean'])

# Conversion to dataframe
X = pd.DataFrame(X.toarray())
# Renaming columns according to words
X.columns = word_names


X = X.values.astype(np.float32)

In [ ]:
predictions = clf.predict(X)

In [ ]:
np.unique(predictions, return_counts=True)

(array([0, 1], dtype=int32), array([17442, 17647]))

In [ ]:
np.unique(y, return_counts=True)

(array([0, 1], dtype=int32), array([7523, 7388]))

In [ ]:
y_pred = [1 if i == 0 else 2 for i in predictions]

In [ ]:
with open("/content/pred_test_nn_v2.txt", "w") as file:
    for element in y_pred:
        file.write(str(element) + "\n")

## NN with Cross Val Score

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neural_network import MLPClassifier
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

clf = MLPClassifier(solver='adam', alpha=1e-5,
                    hidden_layer_sizes=(10, 10, 5, 2),
                    random_state=42, verbose=True, early_stopping=True)

#cross_val_score(clf, X, y, cv=6, verbose=True, n_jobs=2)
clf.fit(X_train, y_train)

Iteration 1, loss = 0.72978341
Validation score: 0.497019
Iteration 2, loss = 0.64229946
Validation score: 0.833085
Iteration 3, loss = 0.49320874
Validation score: 0.871088
Iteration 4, loss = 0.34749010
Validation score: 0.875559
Iteration 5, loss = 0.23093401
Validation score: 0.888972
Iteration 6, loss = 0.16834289
Validation score: 0.875559
Iteration 7, loss = 0.13976788
Validation score: 0.865872
Iteration 8, loss = 0.12316903
Validation score: 0.855440
Iteration 9, loss = 0.11250776
Validation score: 0.856185
Iteration 10, loss = 0.10498269
Validation score: 0.856930
Iteration 11, loss = 0.09900578
Validation score: 0.852459
Iteration 12, loss = 0.09390524
Validation score: 0.852459
Iteration 13, loss = 0.08937915
Validation score: 0.853204
Iteration 14, loss = 0.08530357
Validation score: 0.853204
Iteration 15, loss = 0.08156828
Validation score: 0.852459
Iteration 16, loss = 0.07810986
Validation score: 0.855440
Validation score did not improve more than tol=0.000100 for 10 co

MLPClassifier(alpha=1e-05, early_stopping=True,
              hidden_layer_sizes=(10, 10, 5, 2), random_state=42, verbose=True)

In [ ]:
clf.score(X_test, y_test)

0.8900804289544236

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = vec.transform(test['review_clean'])

# Conversion to dataframe
X = pd.DataFrame(X.toarray())
# Renaming columns according to words
X.columns = word_names


X = X.values.astype(np.float32)

In [ ]:
predictions = clf.predict(X)

In [ ]:
np.unique(predictions, return_counts=True)

(array([0, 1], dtype=int32), array([18609, 16480]))

In [ ]:
np.unique(y, return_counts=True)

(array([0, 1], dtype=int32), array([7523, 7388]))

In [ ]:
y_pred = [1 if i == 0 else 2 for i in predictions]

In [ ]:
with open("/content/pred_test_nn_cv.txt", "w") as file:
    for element in y_pred:
        file.write(str(element) + "\n")

## NN with GS

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neural_network import MLPClassifier
import numpy as np
import pickle

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

parameter_space = {
    'hidden_layer_sizes': [(10,2), (5,2), (25, 10, 2)],
    'activation': ['relu'],#, 'tanh', 'logistic'],
    'solver': ['adam'],
    'alpha': [0.0001],#, 0.001],#, 0.01, 0.1, 1],
    'learning_rate': ['constant', 'adaptive'],
}

clf = GridSearchCV(MLPClassifier(verbose = True, early_stopping=True), parameter_space, n_jobs=-1, cv=4, verbose=4)

clf.fit(X_train, y_train)

# best parameters and score
print('Best parameters found:\n', clf.best_params_)
print('Best score found:\n', clf.best_score_)

Fitting 4 folds for each of 6 candidates, totalling 24 fits
Iteration 1, loss = 0.68060228
Validation score: 0.815201
Iteration 2, loss = 0.62297864
Validation score: 0.849478
Iteration 3, loss = 0.56292523
Validation score: 0.855440
Iteration 4, loss = 0.49193016
Validation score: 0.862891
Iteration 5, loss = 0.41130000
Validation score: 0.859911
Iteration 6, loss = 0.33264789
Validation score: 0.863636
Iteration 7, loss = 0.26504020
Validation score: 0.863636
Iteration 8, loss = 0.21082581
Validation score: 0.860656
Iteration 9, loss = 0.16919468
Validation score: 0.862146
Iteration 10, loss = 0.13678461
Validation score: 0.862891
Iteration 11, loss = 0.11180008
Validation score: 0.863636
Iteration 12, loss = 0.09206418
Validation score: 0.861401
Iteration 13, loss = 0.07630803
Validation score: 0.859165
Iteration 14, loss = 0.06366309
Validation score: 0.859165
Iteration 15, loss = 0.05351776
Validation score: 0.857675
Iteration 16, loss = 0.04516668
Validation score: 0.853949
Itera

In [ ]:
# Test the best model
best_model = clf.best_estimator_

# Save best model
with open('/content/drive/MyDrive/Colab Notebooks/DataScience/best_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

if better than 0.8954

In [ ]:
y_pred = best_model.predict(X_test)

# Evaluate the model and save it
accuracy = np.mean(y_pred == y_test)
print('Accuracy:', accuracy)

Accuracy: 0.8967828418230563


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = vec.transform(test['review_clean'])

# Conversion to dataframe
X = pd.DataFrame(X.toarray())
# Renaming columns according to words
X.columns = word_names


X = X.values.astype(np.float32)

In [ ]:
predictions = best_model.predict(X)

In [ ]:
np.unique(predictions, return_counts=True)

In [ ]:
np.unique(y, return_counts=True)

In [ ]:
y_pred = [1 if i == 0 else 2 for i in predictions]

In [ ]:
with open("/content/pred_test_nn_gs.txt", "w") as file:
    for element in y_pred:
        file.write(str(element) + "\n")

## DT

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

clf = DecisionTreeClassifier(random_state=42, criterion="gini")
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [ ]:
clf.score(X_test, y_test)

0.7091152815013405

In [ ]:
predictions = clf.predict(X)

In [ ]:
np.unique(predictions, return_counts=True)

(array([0, 1], dtype=int32), array([17637, 17452]))

In [ ]:
y_pred = [1 if i == 0 else 2 for i in predictions]

In [ ]:
with open("/content/pred_test_dt.txt", "w") as file:
    for element in y_pred:
        file.write(str(element) + "\n")

In [ ]:
with open("/content/pred_test.txt", "r") as text_file:
    lines = text_file.read().split()

np.unique(lines, return_counts=True)

(array(['1', '2'], dtype='<U1'), array([20298, 14791]))

##Linear SVC

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split

clf = make_pipeline(StandardScaler(),
                    LinearSVC(random_state=42, tol=1e-5, verbose=4))

X_train, X_test, y_train, y_test = train_test_split(
                                    X, y, test_size=0.1, random_state=42)

clf.fit(X_train, y_train)

[LibLinear]

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc',
                 LinearSVC(random_state=42, tol=1e-05, verbose=4))])

In [ ]:
clf.score(X_test, y_test)

0.829088471849866

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = vec.transform(test['review_clean'])

# Conversion to dataframe
X = pd.DataFrame(X.toarray())
# Renaming columns according to words
X.columns = word_names


X = X.values.astype(np.float32)

In [ ]:
predictions = clf.predict(X)

In [ ]:
np.unique(predictions, return_counts=True)

In [ ]:
y_pred = [1 if i == 0 else 2 for i in predictions]

In [ ]:
with open("/content/pred_test_nn.txt", "w") as file:
    for element in y_pred:
        file.write(str(element) + "\n")

## AS A LAST RESORT

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train['review'])
list_tokenized_train = tokenizer.texts_to_sequences(train['review'])

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = train['response']

embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/3
120/120 [==============================] - 56s 426ms/step - loss: 0.5098 - accuracy: 0.7570 - val_loss: 0.3432 - val_accuracy: 0.8522
Epoch 2/3
120/120 [==============================] - 48s 400ms/step - loss: 0.2691 - accuracy: 0.8925 - val_loss: 0.3257 - val_accuracy: 0.8592
Epoch 3/3
120/120 [==============================] - 64s 531ms/step - loss: 0.1801 - accuracy: 0.9346 - val_loss: 0.3668 - val_accuracy: 0.8555


In [ ]:
list_sentences_test = test["review"]
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = model.predict(X_te)

1097/1097 [==============================] - 43s 38ms/step


In [ ]:
y_pred = (prediction > 0.5)

In [ ]:
prediction = [1 if i[0] == 0 else 2 for i in y_pred]

In [ ]:
with open("/content/pred_test_leo.txt", "w") as file:
    for element in prediction:
        file.write(str(element) + "\n")

#huggingFace - score of 383 (reached 92.ish%)

In [ ]:
!pip install transformers evaluate datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataScience/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataScience/test.csv")

In [ ]:
train = train.replace({"response": {1:0, 2:1}})

In [ ]:
from bs4 import BeautifulSoup
from tqdm.auto import tqdm 
tqdm.pandas()


def remove_html(data):
    data = BeautifulSoup(data)
    return data.getText()

train.review = train.review.progress_map(remove_html)
test.review = test.review.progress_map(remove_html)

print(train.head())
test.head()

  0%|          | 0/14911 [00:00<?, ?it/s]

<ipython-input-7-16c9a32dba1c>:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  data = BeautifulSoup(data)


  0%|          | 0/35089 [00:00<?, ?it/s]

   response                                             review
0         1  A wonderful little production. The filming tec...
1         0  Basically there's a family where a little boy ...
2         0  This show was an amazing, fresh & innovative i...
3         0  So im not a big fan of Boll's work but then ag...
4         0  Kind of drawn in by the erotic scenes, only to...


,review
0,"Petter Mattei's ""Love in the Time of Money"" is..."
1,Phil the Alien is one of those quirky films wh...
2,An awful film! It must have been up against so...
3,"First of all, let's get a few things straight ..."
4,"Taut and organically gripping, Edward Dmytryk'..."


In [ ]:
from datasets import Dataset

train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

In [ ]:
train = train.rename_column('review', 'text')
train = train.rename_column('response', 'label')
test = test.rename_column('review', 'text')

In [ ]:
train = train.train_test_split(test_size=0.1)

In [ ]:
train

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 13419
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 1492
    })
})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/13419 [00:00<?, ? examples/s]

Map:   0%|          | 0/1492 [00:00<?, ? examples/s]

Map:   0%|          | 0/35089 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [ ]:
training_args = TrainingArguments(
    output_dir="DSChallenge",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train["train"],
    eval_dataset=tokenized_train["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/leonardosaveri/DSChallenge into local empty directory.


Download file pytorch_model.bin:   0%|          | 1.40k/418M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.50k/3.50k [00:00<?, ?B/s]

Clean file training_args.bin:  29%|##8       | 1.00k/3.50k [00:00<?, ?B/s]

Download file runs/Apr19_15-39-57_4e6d04944665/1681918809.295652/events.out.tfevents.1681918809.4e6d04944665.2…

...

Clean file runs/Apr19_15-39-57_4e6d04944665/events.out.tfevents.1681918809.4e6d04944665.240.4:  15%|#5        …

Clean file pytorch_model.bin:   0%|          | 1.00k/418M [00:00<?, ?B/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.316000,0.194352,0.929625
2,0.149500,0.248087,0.928954


TrainOutput(global_step=1678, training_loss=0.217372718101747, metrics={'train_runtime': 442.5299, 'train_samples_per_second': 60.647, 'train_steps_per_second': 3.792, 'total_flos': 3502346949128064.0, 'train_loss': 0.217372718101747, 'epoch': 2.0})

In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/Apr26_15-13-52_e18e76bbfd71/events.out.tfevents.1682522151.e18e76bbfd71.1761.0:   0%|        …

To https://huggingface.co/leonardosaveri/DSChallenge
   7cfe0b5..3bd630f  main -> main

   7cfe0b5..3bd630f  main -> main

To https://huggingface.co/leonardosaveri/DSChallenge
   3bd630f..6b78473  main -> main

   3bd630f..6b78473  main -> main



'https://huggingface.co/leonardosaveri/DSChallenge/commit/3bd630f2cfa9d480b35ae22b81e74266a6530e38'

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model="leonardosaveri/DSChallenge")

In [ ]:
pred = classifier(test['text'], truncation=True)

In [ ]:
pred

[{'label': 'LABEL_1', 'score': 0.9903265833854675},
 {'label': 'LABEL_0', 'score': 0.8087964057922363},
 {'label': 'LABEL_0', 'score': 0.990757405757904},
 {'label': 'LABEL_0', 'score': 0.9895852208137512},
 {'label': 'LABEL_1', 'score': 0.9749891757965088},
 {'label': 'LABEL_1', 'score': 0.9881086945533752},
 {'label': 'LABEL_0', 'score': 0.9527730345726013},
 {'label': 'LABEL_0', 'score': 0.908545196056366},
 {'label': 'LABEL_1', 'score': 0.9721949100494385},
 {'label': 'LABEL_0', 'score': 0.9650426506996155},
 {'label': 'LABEL_1', 'score': 0.9544132351875305},
 {'label': 'LABEL_0', 'score': 0.9914707541465759},
 {'label': 'LABEL_0', 'score': 0.978954017162323},
 {'label': 'LABEL_1', 'score': 0.97865229845047},
 {'label': 'LABEL_1', 'score': 0.9632110595703125},
 {'label': 'LABEL_0', 'score': 0.6601167321205139},
 {'label': 'LABEL_1', 'score': 0.9574717283248901},
 {'label': 'LABEL_0', 'score': 0.9811323285102844},
 {'label': 'LABEL_1', 'score': 0.9728953242301941},
 {'label': 'LABEL

In [ ]:
pred = [i for i in pred if not isinstance(i, int)]

In [ ]:
pred[-1]

{'label': 'LABEL_0', 'score': 0.9846953749656677}

In [ ]:
y_pred = []
for text in pred:
    if text['label'] == "LABEL_0":
        y_pred.append(1)
    else:
        y_pred.append(2)

In [ ]:
len(y_pred)

35089

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/DataScience/pred_test_hf.txt", "w") as file:
    for element in y_pred:
        file.write(str(element) + "\n")